# HRNet 
Pros: High accuracy, preserves high-resolution details, effective for complex poses. <br>
Cons: Requires significant computational resources.

Approach 

1. Aims to detect 3 keypoints from image I of size W X H X 3 <br>
2. State-of-arts method to caldulate heatmaps which help in finding the location confidence of the k_th keypoint 

download the following libraries 

In [1]:
%pip install dill numpy opencv-python torch torchvision matplotlib yacs


Note: you may need to restart the kernel to use updated packages.


In [2]:
import dill
import math
import os
import logging
import numpy as np
import random
import cv2
import os.path
import pickle
from torch.utils.data import Dataset
from collections import (OrderedDict, namedtuple, defaultdict)
import time
from pathlib import Path
import os.path as osp
import sys
import argparse
import pprint
import shutil
import warnings
from yacs.config import CfgNode as CN
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as functionalF
from torch.autograd import Function
from torch.autograd.function import once_differentiable
from torch.nn.modules.utils import _pair
import torch.utils.data
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision
from torchvision.transforms import functional as F

# a convolution function
def conv3x3(in_planes, out_planes, stride=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=dilation, bias=False, dilation=dilation)

function used above parameters 

1. number of input channels 
2. nunber of output channels 
3. stride - how much filter moves at each step 
4. dilation - allows kernel to cover large area of input without changing parameters 

kernel size is 3x3 

# Basic Block 

a convolution block where we applied convulation using block normalization 

In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1,  downsample=None, dilation=1):
        nn.Module.__init__(self)
        self.conv1 = conv3x3(inplanes, planes, stride, dilation=dilation)
        self.bn1 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.downsample = downsample
        self.stride = stride


    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

# Bottle Neck 

Applying next convulational followed by Batch normalization 

In [4]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1,  downsample=None, dilation=1):
        nn.Module.__init__(self)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=dilation, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

# STN

this the third convulational block i have applied after Bottle Neck CNN 
also called as adaptive represenation transformation 

In [5]:
class STNBLOCK(nn.Module):
    expansion = 1
    def __init__(self, inplanes, outplanes, stride=1, downsample=None, dilation=1, deformable_groups=1):
        nn.Module.__init__(self)
        regular_matrix = torch.tensor(np.array([[-1, -1, -1, 0, 0, 0, 1, 1, 1], \
                                                [-1, 0, 1, -1 ,0 ,1 ,-1, 0, 1]]))
        self.register_buffer('regular_matrix', regular_matrix.float())
        self.downsample = downsample
        self.transform_matrix_conv1 = nn.Conv2d(inplanes, 4, 3, 1, 1, bias=True)
        self.stn_conv1 = DeformConv(inplanes,outplanes, kernel_size=3, stride=stride,padding=dilation, dilation=dilation,deformable_groups=deformable_groups)
        self.bn1 = nn.BatchNorm2d(outplanes, momentum=BN_MOMENTUM)
 
        self.transform_matrix_conv2 = nn.Conv2d(outplanes, 4, 3, 1, 1, bias=True)            
        self.stn_conv2 = DeformConv( outplanes, outplanes, kernel_size=3, stride=1, padding=dilation, dilation=dilation, deformable_groups=deformable_groups)
        self.bn2 = nn.BatchNorm2d(outplanes, momentum=BN_MOMENTUM)
 
        self.relu = nn.ReLU(inplace=True)
 
    def forward(self, x):
        residual = x
        (N,C,H,W) = x.shape
        transform_matrix1 = self.transform_matrix_conv1(x)
        transform_matrix1 = transform_matrix1.permute(0,2,3,1).reshape((N*H*W,2,2))
        offset1 = torch.matmul(transform_matrix1, self.regular_matrix)
        offset1 = offset1-self.regular_matrix
        offset1 = offset1.transpose(1,2)
        offset1 = offset1.reshape((N,H,W,18)).permute(0,3,1,2)
 
        out = self.stn_conv1(x, offset1)
        out = self.bn1(out)
        out = self.relu(out)
 
        transform_matrix2 = self.transform_matrix_conv2(x)
        transform_matrix2 = transform_matrix2.permute(0,2,3,1).reshape((N*H*W,2,2))
        offset2 = torch.matmul(transform_matrix2, self.regular_matrix)
        offset2 = offset2-self.regular_matrix
        offset2 = offset2.transpose(1,2)
        offset2 = offset2.reshape((N,H,W,18)).permute(0,3,1,2)
        out = self.stn_conv2(out, offset2)
        out = self.bn2(out)
        
        if self.downsample is not None:
            residual = self.downsample(x)
 
        out += residual
        out = self.relu(out)
 
        return out

# Basic Module used by HRNet  model 

1. Basic case  - where the number of branches used is 1 
2. Complicated case - where there are more than one branches used hence we fuse these brances

Forward method used -- > multiple branches are used and outputs are fused together 


why ? -- Multiple branches in a network can process features at different scales or resolutions. By fusing the outputs of these branches, the network can integrate features from different scales, which can be beneficial for capturing more comprehensive information about the input.

In [6]:
def forward(self, x):
    if self.num_branches == 1:
        return [self.branches[0](x[0])]

    for i in range(self.num_branches):
        x[i] = self.branches[i](x[i])

    x_fuse = []

    for i in range(len(self.fuse_layers)):
        y = x[0] if i == 0 else self.fuse_layers[i][0](x[0])
        for j in range(1, self.num_branches):
            if i == j:
                y = y + x[j]
            else:
                y = y + self.fuse_layers[i][j](x[j])
        x_fuse.append(self.relu(y))

    return x_fuse

In [7]:
def _make_one_branch(self, branch_index, block, num_blocks, num_channels, stride=1):
    downsample = None
    if stride != 1 or  self.num_inchannels[branch_index] != num_channels[branch_index] * block.expansion:
        downsample = nn.Sequential(
            nn.Conv2d(self.num_inchannels[branch_index], num_channels[branch_index] * block.expansion, kernel_size=1, stride=stride),
            nn.BatchNorm2d(num_channels[branch_index] * block.expansion, momentum=BN_MOMENTUM)
        )

    layers = []
    layers.append(block(self.num_inchannels[branch_index], num_channels[branch_index], stride, downsample))
    self.num_inchannels[branch_index] =  num_channels[branch_index] * block.expansion

    for i in range(1, num_blocks[branch_index]):
        layers.append(block(self.num_inchannels[branch_index], num_channels[branch_index]))

    return nn.Sequential(*layers)

In [ ]:
class HighResolutionModule(nn.Module):
    def __init__(self, num_branches, blocks, num_blocks, num_inchannels, num_channels, fuse_method, multi_scale_output=True):
        nn.Module.__init__(self)
        self._check_branches(num_branches, blocks, num_blocks, num_inchannels, num_channels)

        self.num_inchannels = num_inchannels
        self.fuse_method = fuse_method
        self.num_branches = num_branches

        self.multi_scale_output = multi_scale_output

        self.branches = self._make_branches( num_branches, blocks, num_blocks, num_channels)
        self.fuse_layers = self._make_fuse_layers()
        self.relu = nn.ReLU(True)

    def _check_branches(self, num_branches, blocks, num_blocks,
                        num_inchannels, num_channels):
        if num_branches != len(num_blocks):
            error_msg = 'NUM_BRANCHES({}) <> NUM_BLOCKS({})'.format(  num_branches, len(num_blocks))
            raise ValueError(error_msg)

        if num_branches != len(num_channels):
            error_msg = 'NUM_BRANCHES({}) <> NUM_CHANNELS({})'.format(num_branches, len(num_channels))
            raise ValueError(error_msg)

        if num_branches != len(num_inchannels):
            error_msg = 'NUM_BRANCHES({}) <> NUM_INCHANNELS({})'.format( num_branches, len(num_inchannels))
            raise ValueError(error_msg)

    def _make_one_branch(self, branch_index, block, num_blocks, num_channels, stride=1):
        downsample = None
        if stride != 1 or  self.num_inchannels[branch_index] != num_channels[branch_index] * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.num_inchannels[branch_index], num_channels[branch_index] * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(num_channels[branch_index] * block.expansion, momentum=BN_MOMENTUM)
            )

        layers = []
        layers.append(block(self.num_inchannels[branch_index], num_channels[branch_index], stride, downsample))
        self.num_inchannels[branch_index] =  num_channels[branch_index] * block.expansion
        
        for i in range(1, num_blocks[branch_index]):
            layers.append(block(self.num_inchannels[branch_index], num_channels[branch_index]))

        return nn.Sequential(*layers)

    def _make_branches(self, num_branches, block, num_blocks, num_channels):
        branches = []

        for i in range(num_branches):
            branches.append(self._make_one_branch(i, block, num_blocks, num_channels))

        return nn.ModuleList(branches)

    def _make_fuse_layers(self):
        if self.num_branches == 1:
            return None

        num_branches = self.num_branches
        num_inchannels = self.num_inchannels
        fuse_layers = []
        for i in range(num_branches if self.multi_scale_output else 1):
            fuse_layer = []
            for j in range(num_branches):
                if j > i:
                    fuse_layer.append(nn.Sequential(
                        nn.Conv2d(num_inchannels[j], num_inchannels[i],1,1,0, bias=False),
                        nn.BatchNorm2d(num_inchannels[i]),
                        nn.Upsample(scale_factor=2**(j-i), mode='nearest')))
                elif j == i:
                    fuse_layer.append(None)
                else:
                    conv3x3s = []
                    for k in range(i-j):
                        if k == i - j - 1:
                            num_outchannels_conv3x3 = num_inchannels[i]
                            conv3x3s.append(nn.Sequential(
                                nn.Conv2d(num_inchannels[j],num_outchannels_conv3x3, 3, 2, 1, bias=False),
                                nn.BatchNorm2d(num_outchannels_conv3x3)))
                        else:
                            num_outchannels_conv3x3 = num_inchannels[j]
                            conv3x3s.append(nn.Sequential(
                                nn.Conv2d(num_inchannels[j], num_outchannels_conv3x3, 3, 2, 1, bias=False),
                                nn.BatchNorm2d(num_outchannels_conv3x3),
                                nn.ReLU(True)))
                    fuse_layer.append(nn.Sequential(*conv3x3s))
            fuse_layers.append(nn.ModuleList(fuse_layer))

        return nn.ModuleList(fuse_layers)

    def get_num_inchannels(self):
        return self.num_inchannels

    def forward(self, x):
        if self.num_branches == 1:
            return [self.branches[0](x[0])]

        for i in range(self.num_branches):
            x[i] = self.branches[i](x[i])

        x_fuse = []

        for i in range(len(self.fuse_layers)):
            y = x[0] if i == 0 else self.fuse_layers[i][0](x[0])
            for j in range(1, self.num_branches):
                if i == j:
                    y = y + x[j]
                else:
                    y = y + self.fuse_layers[i][j](x[j])
            x_fuse.append(self.relu(y))

        return x_fuse

In [8]:
# common params for NETWORK
cfgMODEL = CN()
cfgMODEL.NAME = 'pose_hrnet'
cfgMODEL.INIT_WEIGHTS = True
cfgMODEL.PRETRAINED = '' # 'model/imagenet/hrnet_w32-36af842e.pth'
cfgMODEL.NUM_JOINTS = 15 # 14
cfgMODEL.EXTRA = CN(new_allowed=True)

cfgMODEL.EXTRA.FINAL_CONV_KERNEL = 1
cfgMODEL.EXTRA.PRETRAINED_LAYERS = ['*']
cfgMODEL.EXTRA.STEM_INPLANES =  64

cfgMODEL.EXTRA.STAGE2 = CN(new_allowed=True)
cfgMODEL.EXTRA.STAGE2.NUM_MODULES = 1
cfgMODEL.EXTRA.STAGE2.NUM_BRANCHES = 2
cfgMODEL.EXTRA.STAGE2.BLOCK = 'BASIC'
cfgMODEL.EXTRA.STAGE2.NUM_BLOCKS = [4, 4]
cfgMODEL.EXTRA.STAGE2.NUM_CHANNELS = [32, 64]
cfgMODEL.EXTRA.STAGE2.FUSE_METHOD = 'SUM'

cfgMODEL.EXTRA.STAGE3 = CN(new_allowed=True)
cfgMODEL.EXTRA.STAGE3.NUM_MODULES = 4
cfgMODEL.EXTRA.STAGE3.NUM_BRANCHES = 3
cfgMODEL.EXTRA.STAGE3.BLOCK = 'BASIC'
cfgMODEL.EXTRA.STAGE3.NUM_BLOCKS = [4, 4, 4]
cfgMODEL.EXTRA.STAGE3.NUM_CHANNELS = [32, 64, 128]
cfgMODEL.EXTRA.STAGE3.FUSE_METHOD = 'SUM'

cfgMODEL.EXTRA.STAGE4 = CN(new_allowed=True)
cfgMODEL.EXTRA.STAGE4.NUM_MODULES = 3
cfgMODEL.EXTRA.STAGE4.NUM_BRANCHES = 4
cfgMODEL.EXTRA.STAGE4.BLOCK = 'BASIC'
cfgMODEL.EXTRA.STAGE4.NUM_BLOCKS = [4, 4, 4, 4]
cfgMODEL.EXTRA.STAGE4.NUM_CHANNELS = [32, 64, 128, 256]
cfgMODEL.EXTRA.STAGE4.FUSE_METHOD = 'SUM'

cfgMODEL.EXTRA.DECONV = CN(new_allowed=True)
cfgMODEL.EXTRA.DECONV.NUM_DECONVS = 0
cfgMODEL.EXTRA.DECONV.NUM_CHANNELS = [32]
cfgMODEL.EXTRA.DECONV.KERNEL_SIZE = 4
cfgMODEL.EXTRA.DECONV.NUM_BASIC_BLOCKS = 0
cfgMODEL.EXTRA.DECONV.CAT_OUTPUT = True

cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_HEATMAP = CN(new_allowed=True)
cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_HEATMAP.BLOCK = ['STNBLOCK','STNBLOCK','STNBLOCK']
cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_HEATMAP.NUM_BLOCKS = [1,1,1]
cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_HEATMAP.NUM_CHANNELS = [32,32,32]
cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_HEATMAP.DILATION_RATE = [1,2,3]

cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_REGRESSION = CN(new_allowed=True)
cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_REGRESSION.BLOCK = ['STNBLOCK']
cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_REGRESSION.NUM_BLOCKS = [1]
cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_REGRESSION.NUM_CHANNELS = [256]
cfgMODEL.EXTRA.MULTI_LEVEL_OUTPUT_REGRESSION.DILATION_RATE = [1]

In [9]:
def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.layer1(x)

    x_list = []
    for i in range(self.stage2_cfg['NUM_BRANCHES']):
        if self.transition1[i] is not None:
            x_list.append(self.transition1[i](x))
        else:
            x_list.append(x)
    y_list = self.stage2(x_list)

    x_list = []
    for i in range(self.stage3_cfg['NUM_BRANCHES']):
        if self.transition2[i] is not None:
            x_list.append(self.transition2[i](y_list[-1]))
        else:
            x_list.append(y_list[i])
    y_list = self.stage3(x_list)

    x_list = []
    for i in range(self.stage4_cfg['NUM_BRANCHES']):
        if self.transition3[i] is not None:
            x_list.append(self.transition3[i](y_list[-1]))
        else:
            x_list.append(y_list[i])
    x = self.stage4(x_list)

    # Upsampling
    x0_h, x0_w = x[0].size(2), x[0].size(3)

    x = torch.cat([x[0], \
        functionalF.upsample(x[1],size=(x0_h, x0_w), mode='bilinear'), \
        functionalF.upsample(x[2], size=(x0_h, x0_w), mode='bilinear'), \
        functionalF.upsample(x[3], size=(x0_h, x0_w), mode='bilinear')], 1)

    final_outputs = []
    final_offsets = []

    final_output = []
    final_offset = []

    for j in range(len(self.multi_level_layers_4x_heatmap)):
        final_output.append(self.final_layers[0](self.multi_level_layers_4x_heatmap[j](self.transition_cls(x))))

    x = torch.cat([x, torch.mean(torch.stack(final_output), 0)], 1)

    for j in range(len(self.multi_level_layers_4x_regression)):
        final_offset.append(self.final_layers[1](self.multi_level_layers_4x_regression[j](self.transition_reg(x))))

    for i in range(len(final_output)):
        final_output[i] = torch.cat([final_output[i], final_offset[0][:,-1:,:,:]], 1)

    final_outputs.append([torch.mean(torch.stack(final_output), 0)])
    final_offsets.append([final_offset[0][:,:-1,:,:]])

    return final_outputs, final_offsets

In [ ]:
class PoseHigherResolutionNet(nn.Module):
    def __init__(self, cfgMODEL, cfgDATASET,cfgLOSS, **kwargs):
        self.inplanes = 64
        self.dim_heat = cfgDATASET.NUM_JOINTS-1 if cfgDATASET.WITH_CENTER else cfgDATASET.NUM_JOINTS
        self.dim_reg = self.dim_heat * 2 + 1
        extra = cfgMODEL.EXTRA
        nn.Module.__init__(self)

        # stem net
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64, momentum=BN_MOMENTUM)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1,  bias=False)
        self.bn2 = nn.BatchNorm2d(64, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(Bottleneck, 64, 64, 4)

        self.stage2_cfg = cfgMODEL['EXTRA']['STAGE2']
        num_channels = self.stage2_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage2_cfg['BLOCK']]
        num_channels = [ num_channels[i] * block.expansion for i in range(len(num_channels))]
        self.transition1 = self._make_transition_layer([256], num_channels)
        self.stage2, pre_stage_channels = self._make_stage( self.stage2_cfg, num_channels)

        self.stage3_cfg = cfgMODEL['EXTRA']['STAGE3']
        num_channels = self.stage3_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage3_cfg['BLOCK']]
        num_channels = [ num_channels[i] * block.expansion for i in range(len(num_channels)) ]
        self.transition2 = self._make_transition_layer( pre_stage_channels, num_channels)
        self.stage3, pre_stage_channels = self._make_stage( self.stage3_cfg, num_channels)

        self.stage4_cfg = cfgMODEL['EXTRA']['STAGE4']
        num_channels = self.stage4_cfg['NUM_CHANNELS']
        block = blocks_dict[self.stage4_cfg['BLOCK']]
        num_channels = [ num_channels[i] * block.expansion for i in range(len(num_channels)) ]
        self.transition3 = self._make_transition_layer( pre_stage_channels, num_channels)
        self.stage4, pre_stage_channels = self._make_stage( self.stage4_cfg, num_channels, multi_scale_output=True)
        
        inp_channels = np.int(np.sum(pre_stage_channels))
        multi_output_config_heatmap = cfgMODEL['EXTRA']['MULTI_LEVEL_OUTPUT_HEATMAP']
        multi_output_config_regression = cfgMODEL['EXTRA']['MULTI_LEVEL_OUTPUT_REGRESSION']
        self.transition_cls = nn.Sequential(
                                nn.Conv2d(inp_channels,multi_output_config_heatmap['NUM_CHANNELS'][0], 1, 1, 0, bias=False),
                                nn.BatchNorm2d(multi_output_config_heatmap['NUM_CHANNELS'][0]),
                                nn.ReLU(True))
        self.transition_reg = nn.Sequential(
                                nn.Conv2d(inp_channels + self.dim_heat, multi_output_config_regression['NUM_CHANNELS'][0],  1, 1, 0, bias=False),
                                nn.BatchNorm2d(multi_output_config_regression['NUM_CHANNELS'][0]),
                                nn.ReLU(True))
        self.multi_level_layers_4x_heatmap = self._make_multi_level_layer( multi_output_config_heatmap)
        self.multi_level_layers_4x_regression = self._make_multi_level_layer( multi_output_config_regression)
        
        self.final_layers = self._make_final_layers(  cfgMODEL,cfgDATASET, multi_output_config_heatmap, multi_output_config_regression)
        #self.deconv_layers = self._make_deconv_layers( cfgMODEL, pre_stage_channels[0])

        self.num_deconvs = extra.DECONV.NUM_DECONVS
        self.deconv_config = cfgMODEL.EXTRA.DECONV
        self.loss_config = cfgLOSS

        self.pretrained_layers = cfgMODEL['EXTRA']['PRETRAINED_LAYERS']

    def _make_final_layers(self, cfgMODEL,cfgDATASET, multi_output_config_heatmap, multi_output_config_regression):
        extra = cfgMODEL.EXTRA

        final_layers = []
        final_layers.append(nn.Conv2d(in_channels=multi_output_config_heatmap['NUM_CHANNELS'][0],out_channels=self.dim_heat,
            kernel_size=extra.FINAL_CONV_KERNEL,stride=1,padding=1 if extra.FINAL_CONV_KERNEL == 3 else 0 ))

        # for regression
        if cfgDATASET.OFFSET_REG:
            final_layers.append(nn.Conv2d(
                in_channels=multi_output_config_regression['NUM_CHANNELS'][0], out_channels=self.dim_reg,
                kernel_size=extra.FINAL_CONV_KERNEL, stride=1, padding=1 if extra.FINAL_CONV_KERNEL == 3 else 0))

        return nn.ModuleList(final_layers)

    def _make_layer( self, block, inplanes, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=BN_MOMENTUM))

        layers = []
        layers.append(block(inplanes, planes, stride, downsample, dilation=dilation))
        inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(inplanes, planes, dilation=dilation))

        return nn.Sequential(*layers)

    def _make_multi_level_layer(self, layer_config):
        multi_level_layers = []

        first_branch = self._make_layer(
                blocks_dict[layer_config['BLOCK'][0]],
                layer_config['NUM_CHANNELS'][0],
                layer_config['NUM_CHANNELS'][0],
                layer_config['NUM_BLOCKS'][0],
                dilation=layer_config['DILATION_RATE'][0]
        )
        multi_level_layers.append(first_branch)
        
        for i, d in enumerate(layer_config['DILATION_RATE'][1:]):
            branch = self._make_layer(
                blocks_dict[layer_config['BLOCK'][i+1]],
                layer_config['NUM_CHANNELS'][i+1],
                layer_config['NUM_CHANNELS'][i+1],
                layer_config['NUM_BLOCKS'][i+1],
                dilation=d
            )

            for module in zip(first_branch.named_modules(), branch.named_modules()):
                if 'conv' in module[0][0] and 'conv' in module[1][0]:
                    module[1][1].weight = module[0][1].weight

            multi_level_layers.append(branch)

        return nn.ModuleList(multi_level_layers)

    def _make_deconv_layers(self, cfgMODEL, input_channels):
        dim_tag = cfgMODEL.NUM_JOINTS
        extra = cfgMODEL.EXTRA
        deconv_cfg = extra.DECONV

        deconv_layers = []
        for i in range(deconv_cfg.NUM_DECONVS):
            if deconv_cfg.CAT_OUTPUT[i]:
                final_output_channels = cfgMODEL.NUM_JOINTS + dim_tag  if cfgLOSS.WITH_AE_LOSS[i] else cfgMODEL.NUM_JOINTS
                input_channels += final_output_channels
            output_channels = deconv_cfg.NUM_CHANNELS[i]
            deconv_kernel, padding, output_padding =  self._get_deconv_cfg(deconv_cfg.KERNEL_SIZE[i])

            layers = []
            layers.append(nn.Sequential(
                nn.ConvTranspose2d( in_channels=input_channels,out_channels=output_channels,
                    kernel_size=deconv_kernel,stride=2, padding=padding,output_padding=output_padding, bias=False),
                nn.BatchNorm2d(output_channels, momentum=BN_MOMENTUM),
                nn.ReLU(inplace=True)
            ))
            for _ in range(cfgMODEL.EXTRA.DECONV.NUM_BASIC_BLOCKS):
                layers.append(nn.Sequential( BasicBlock(output_channels, output_channels)))
            deconv_layers.append(nn.Sequential(*layers))
            input_channels = output_channels

        return nn.ModuleList(deconv_layers)

    def _get_deconv_cfg(self, deconv_kernel):
        if deconv_kernel == 4:
            padding = 1
            output_padding = 0
        elif deconv_kernel == 3:
            padding = 1
            output_padding = 1
        elif deconv_kernel == 2:
            padding = 0
            output_padding = 0

        return deconv_kernel, padding, output_padding

    def _make_transition_layer(self, num_channels_pre_layer, num_channels_cur_layer):
        num_branches_cur = len(num_channels_cur_layer)
        num_branches_pre = len(num_channels_pre_layer)

        transition_layers = []
        for i in range(num_branches_cur):
            if i < num_branches_pre:
                if num_channels_cur_layer[i] != num_channels_pre_layer[i]:
                    transition_layers.append(nn.Sequential(
                        nn.Conv2d(num_channels_pre_layer[i], num_channels_cur_layer[i], 3, 1, 1, bias=False),
                        nn.BatchNorm2d(num_channels_cur_layer[i]),
                        nn.ReLU(inplace=True)))
                else:
                    transition_layers.append(None)
            else:
                conv3x3s = []
                for j in range(i+1-num_branches_pre):
                    inchannels = num_channels_pre_layer[-1]
                    outchannels = num_channels_cur_layer[i] if j == i-num_branches_pre else inchannels
                    conv3x3s.append(nn.Sequential(
                        nn.Conv2d(inchannels, outchannels, 3, 2, 1, bias=False),
                        nn.BatchNorm2d(outchannels),
                        nn.ReLU(inplace=True)))
                transition_layers.append(nn.Sequential(*conv3x3s))

        return nn.ModuleList(transition_layers)

    def _make_stage(self, layer_config, num_inchannels,
                    multi_scale_output=True):
        num_modules = layer_config['NUM_MODULES']
        num_branches = layer_config['NUM_BRANCHES']
        num_blocks = layer_config['NUM_BLOCKS']
        num_channels = layer_config['NUM_CHANNELS']
        block = blocks_dict[layer_config['BLOCK']]
        fuse_method = layer_config['FUSE_METHOD']

        modules = []
        for i in range(num_modules):
            # multi_scale_output is only used last module
            if not multi_scale_output and i == num_modules - 1:
                reset_multi_scale_output = False
            else:
                reset_multi_scale_output = True

            modules.append(HighResolutionModule(num_branches,block,num_blocks,num_inchannels,num_channels,fuse_method, reset_multi_scale_output) )
            num_inchannels = modules[-1].get_num_inchannels()

        return nn.Sequential(*modules), num_inchannels

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.layer1(x)

        x_list = []
        for i in range(self.stage2_cfg['NUM_BRANCHES']):
            if self.transition1[i] is not None:
                x_list.append(self.transition1[i](x))
            else:
                x_list.append(x)
        y_list = self.stage2(x_list)

        x_list = []
        for i in range(self.stage3_cfg['NUM_BRANCHES']):
            if self.transition2[i] is not None:
                x_list.append(self.transition2[i](y_list[-1]))
            else:
                x_list.append(y_list[i])
        y_list = self.stage3(x_list)

        x_list = []
        for i in range(self.stage4_cfg['NUM_BRANCHES']):
            if self.transition3[i] is not None:
                x_list.append(self.transition3[i](y_list[-1]))
            else:
                x_list.append(y_list[i])
        x = self.stage4(x_list)

        # Upsampling
        x0_h, x0_w = x[0].size(2), x[0].size(3)

        x = torch.cat([x[0], \
            functionalF.upsample(x[1],size=(x0_h, x0_w), mode='bilinear'), \
            functionalF.upsample(x[2], size=(x0_h, x0_w), mode='bilinear'), \
            functionalF.upsample(x[3], size=(x0_h, x0_w), mode='bilinear')], 1)

        final_outputs = []
        final_offsets = []

        final_output = []
        final_offset = []

        for j in range(len(self.multi_level_layers_4x_heatmap)):
            final_output.append(self.final_layers[0](self.multi_level_layers_4x_heatmap[j](self.transition_cls(x))))

        x = torch.cat([x, torch.mean(torch.stack(final_output), 0)], 1)
    
        for j in range(len(self.multi_level_layers_4x_regression)):
            final_offset.append(self.final_layers[1](self.multi_level_layers_4x_regression[j](self.transition_reg(x))))

        for i in range(len(final_output)):
            final_output[i] = torch.cat([final_output[i], final_offset[0][:,-1:,:,:]], 1)
        
        final_outputs.append([torch.mean(torch.stack(final_output), 0)])
        final_offsets.append([final_offset[0][:,:-1,:,:]])

        return final_outputs, final_offsets

    def init_weights(self, pretrained='', verbose=True):
        print('=> init weights from normal distribution')
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.001)
                for name, _ in m.named_parameters():
                    if name in ['bias']:
                        nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.ConvTranspose2d):
                nn.init.normal_(m.weight, std=0.001)
                for name, _ in m.named_parameters():
                    if name in ['bias']:
                        nn.init.constant_(m.bias, 0)

        for m in self.modules():
            if hasattr(m, 'conv_mask_offset_1'):
                nn.init.constant_(m.conv_mask_offset_1.weight, 0)
                nn.init.constant_(m.conv_mask_offset_1.bias, 0)
            if hasattr(m, 'conv_mask_offset_2'):
                nn.init.constant_(m.conv_mask_offset_2.weight, 0)
                nn.init.constant_(m.conv_mask_offset_2.bias, 0)
            if hasattr(m, 'transform_matrix_conv1'):
                nn.init.constant_(m.transform_matrix_conv1.weight, 0)
                nn.init.constant_(m.transform_matrix_conv1.bias, 0)
            if hasattr(m, 'transform_matrix_conv2'):
                nn.init.constant_(m.transform_matrix_conv2.weight, 0)
                nn.init.constant_(m.transform_matrix_conv2.bias, 0)

        parameters_names = set()
        for name, _ in self.named_parameters():
            parameters_names.add(name)

        buffers_names = set()
        for name, _ in self.named_buffers():
            buffers_names.add(name)

        if os.path.isfile(pretrained):
            pretrained_state_dict = torch.load(pretrained, 
                            map_location=lambda storage, loc: storage)
            print('=> loading pretrained model {}'.format(pretrained))

            need_init_state_dict = {}
            for name, m in pretrained_state_dict.items():
                if name.split('.')[0] in self.pretrained_layers \
                   or self.pretrained_layers[0] is '*':
                    if name in parameters_names or name in buffers_names:
                        if verbose:
                            print('=> init {} from {}'.format(name, pretrained))
                        need_init_state_dict[name] = m
            self.load_state_dict(need_init_state_dict, strict=False)

In [10]:
with open('model_funcs.pkl', 'wb') as f:
    dill.dump(cfgMODEL, f)
    dill.dump(conv3x3, f)
    dill.dump(BasicBlock, f, byref=False)
    dill.dump(Bottleneck, f, byref=False)
    dill.dump(STNBLOCK, f, byref=False)
    dill.dump(HighResolutionModule, f, byref=False)
    dill.dump(PoseHigherResolutionNet, f, byref=False)

NameError: name 'HighResolutionModule' is not defined